In [1]:
a={'Chinook': {'dbo': {'tables': {'Album': {'AlbumId': 'INTEGER'}, 'Artist': {'ArtistId': 'INTEGER'}, 'Customer': {'Address': 'NVARCHAR(70) COLLATE "SQL_Latin1_General_CP1_CI_AS"'}, 'Employee': {'Address': 'NVARCHAR(70) COLLATE "SQL_Latin1_General_CP1_CI_AS"'}, 'Genre': {'GenreId': 'INTEGER'}, 'Invoice': {'BillingAddress': 'NVARCHAR(70) COLLATE "SQL_Latin1_General_CP1_CI_AS"'}, 'InvoiceLine': {'InvoiceId': 'INTEGER'}, 'MediaType': {'MediaTypeId': 'INTEGER'}, 'Playlist': {'Name': 'NVARCHAR(120) COLLATE "SQL_Latin1_General_CP1_CI_AS"'}, 'PlaylistTrack': {'PlaylistId': 'INTEGER'}, 'Track': {'AlbumId': 'INTEGER'}}}}}

In [5]:
a['Chinook']['dbo']['tables'].keys()

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track'])

In [7]:
import json
with open("JSON\db_details.json", 'r') as f:
    db_creds = json.load(f)

In [18]:
import pymssql
from urllib.parse import quote_plus
db_user=db_creds['db_user']
db_password=db_creds['db_password']
db_host=db_creds['db_host']
db_port='1433'
db_name=db_creds['db_name']
print(db_user,db_password,db_host,db_port,db_name)
#tables=eval(db_creds['tables'])
encoded_password = quote_plus(db_password)
conn = pymssql.connect(database=db_name,host=db_host,user=db_user,password=encoded_password,as_dict=True,port=db_port)

mihir mihir MIHIR-G15 1433 Chinook


In [20]:
import pandas as pd

sql_query='SELECT il.InvoiceLineId, t.Name AS TrackName, ar.Name AS ArtistName FROM dbo.InvoiceLine il JOIN dbo.Track t ON il.TrackId = t.TrackId JOIN dbo.Album al ON t.AlbumId = al.AlbumId JOIN dbo.Artist ar ON al.ArtistId = ar.ArtistId;'
df = pd.read_sql(sql_query, conn)

C:\Users\mihir\AppData\Local\Temp\ipykernel_23864\95255237.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [43]:
from langchain_community.chat_models import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-j3LbRtzWZr7BskBM0z7JT3BlbkFJWjUf2y5rsnMqZGv44O49"
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.1, model=llm_model)

prompt_working='''Based on the table schema, write a clean MS SQL query(without mentioning sql in the begining) that would answer the user's question. Use the TOP function instead of LIMIT if it is needed.The format of the schema is in json format {"Table Name":{"Column Name":'data type'}}
TABLE DESCRIPTION:
{'dbo.Album': {'AlbumId': 'int', 'ArtistId': 'int', 'Title': 'nvarchar'}, 'dbo.Artist': {'ArtistId': 'int', 'Name': 'nvarchar'}, 'dbo.Customer': {'Address': 'nvarchar', 'City': 'nvarchar', 'Company': 'nvarchar', 'Country': 'nvarchar', 'CustomerId': 'int', 'Email': 'nvarchar', 'Fax': 'nvarchar', 'FirstName': 'nvarchar', 'LastName': 'nvarchar', 'Phone': 'nvarchar', 'PostalCode': 'nvarchar', 'State': 'nvarchar', 'SupportRepId': 'int'}, 'dbo.Employee': {'Address': 'nvarchar', 'BirthDate': 'datetime', 'City': 'nvarchar', 'Country': 'nvarchar', 'Email': 'nvarchar', 'EmployeeId': 'int', 'Fax': 'nvarchar', 'FirstName': 'nvarchar', 'HireDate': 'datetime', 'LastName': 'nvarchar', 'Phone': 'nvarchar', 'PostalCode': 'nvarchar', 'ReportsTo': 'int', 'State': 'nvarchar', 'Title': 'nvarchar'}, 'dbo.Genre': {'GenreId': 'int', 'Name': 'nvarchar'}, 'dbo.Invoice': {'BillingAddress': 'nvarchar', 'BillingCity': 'nvarchar', 'BillingCountry': 'nvarchar', 'BillingPostalCode': 'nvarchar', 'BillingState': 'nvarchar', 'CustomerId': 'int', 'InvoiceDate': 'datetime', 'InvoiceId': 'int', 'Total': 'numeric'}, 'dbo.InvoiceLine': {'InvoiceId': 'int', 'InvoiceLineId': 'int', 'Quantity': 'int', 'TrackId': 'int', 'UnitPrice': 'numeric'}, 'dbo.MediaType': {'MediaTypeId': 'int', 'Name': 'nvarchar'}, 'dbo.Playlist': {'Name': 'nvarchar', 'PlaylistId': 'int'}, 'dbo.PlaylistTrack': {'PlaylistId': 'int', 'TrackId': 'int'}, 'dbo.Track': {'AlbumId': 'int', 'Bytes': 'int', 'Composer': 'nvarchar', 'GenreId': 'int', 'MediaTypeId': 'int', 'Milliseconds': 'int', 'Name': 'nvarchar', 'TrackId': 'int', 'UnitPrice': 'numeric'}}    

QUESTION:
Provide a query that shows the number of customers assigned to each sales agent.

ANSWER:

'''
prompt='''Based on the table schema, write a clean MS SQL query(without mentioning sql in the begining) that would answer the user's question.
Use the TOP function instead of LIMIT if it is needed.
The format of the schema is in json format {"Table Name":{"Column Name":'data type'}}

TABLE DESCRIPTION:
{'dbo.Album': {'AlbumId': 'int', 'Title': 'nvarchar', 'ArtistId': 'int'}, 'dbo.Artist': {'ArtistId': 'int', 'Name': 'nvarchar'}, 'dbo.Customer': {'CustomerId': 'int', 'FirstName': 'nvarchar', 'LastName': 'nvarchar', 'Company': 'nvarchar', 'Address': 'nvarchar', 'City': 'nvarchar', 'State': 'nvarchar', 'Country': 'nvarchar', 'PostalCode': 'nvarchar', 'Phone': 'nvarchar', 'Fax': 'nvarchar', 'Email': 'nvarchar', 'SupportRepId': 'int'}, 'dbo.Employee': {'EmployeeId': 'int', 'LastName': 'nvarchar', 'FirstName': 'nvarchar', 'Title': 'nvarchar', 'ReportsTo': 'int', 'BirthDate': 'datetime', 'HireDate': 'datetime', 'Address': 'nvarchar', 'City': 'nvarchar', 'State': 'nvarchar', 'Country': 'nvarchar', 'PostalCode': 'nvarchar', 'Phone': 'nvarchar', 'Fax': 'nvarchar', 'Email': 'nvarchar'}, 'dbo.Genre': {'GenreId': 'int', 'Name': 'nvarchar'}, 'dbo.Invoice': {'InvoiceId': 'int', 'CustomerId': 'int', 'InvoiceDate': 'datetime', 'BillingAddress': 'nvarchar', 'BillingCity': 'nvarchar', 'BillingState': 'nvarchar', 'BillingCountry': 'nvarchar', 'BillingPostalCode': 'nvarchar', 'Total': 'numeric'}, 'dbo.InvoiceLine': {'InvoiceLineId': 'int', 'InvoiceId': 'int', 'TrackId': 'int', 'UnitPrice': 'numeric', 'Quantity': 'int'}, 'dbo.MediaType': {'MediaTypeId': 'int', 'Name': 'nvarchar'}, 'dbo.Playlist': {'PlaylistId': 'int', 'Name': 'nvarchar'}, 'dbo.PlaylistTrack': {'PlaylistId': 'int', 'TrackId': 'int'}, 'dbo.Track': {'TrackId': 'int', 'Name': 'nvarchar', 'AlbumId': 'int', 'MediaTypeId': 'int', 'GenreId': 'int', 'Composer': 'nvarchar', 'Milliseconds': 'int', 'Bytes': 'int', 'UnitPrice': 'numeric'}}        

QUESTION:
Provide a query that shows the number of customers assigned to each sales agent.

ANSWER:'''


In [44]:
Sql_ans = model.invoke(prompt)

In [45]:
Sql_ans.content

'SELECT SupportRepId, COUNT(CustomerId) AS NumberOfCustomers\nFROM Customer\nGROUP BY SupportRepId;'

In [46]:
Sql_ans = llm.invoke(prompt_working)

In [47]:
Sql_ans.content

'SELECT SupportRepId, COUNT(CustomerId) AS NumberOfCustomers\nFROM dbo.Customer\nGROUP BY SupportRepId;'

In [48]:
pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.0.0
accelerate                               0.29.1
aiofiles                                 23.2.1
aiohttp                                  3.9.3
aiosignal                                1.3.1
albumentations                           1.3.1
alembic                                  1.13.1
aliyun-python-sdk-core                   2.14.0
aliyun-python-sdk-kms                    2.16.2
altair                                   5.2.0
altgraph                                 0.17.3
annotated-types                          0.6.0
antlr4-python3-runtime                   4.9.3
anyio                                    3.7.1
appdirs                                  1.4.4
arrow                                    1.3.0
asgiref                                  3.8.1
aspose-pdf                               24.1.0
asttokens                            